In [1]:
import pandas as pd
import json
# import numpy as np
from os import listdir
from ete3 import Tree


In [8]:
#### get list of files for relax folder ####
path="/home/maeperez/scratch/Clams/hyphy2.5/absrel"
path_out="/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data/"

genes=[]
for file in listdir(path):
    if file[-5:]==".json":
        genes+=[str(file.split('.')[0])]

genes=sorted(genes)

pval_threshold=0.01

#### init all dictionaries and functions ####
branches_dic={'Bathy_MS2': 'Bathy',
 'SUP05_SP6': 'SUP05',
 'R_fausta_': 'R.fausta',
 'R_magnifi': 'R.magnifica',
 'R_pacific': 'R.pacifica',
 'R_phaseol': 'R.phaseoliformis',
 'R_pliocar': 'R.pliocardia',
 'R_rectima': 'R.rectimargo',
 'R_southwa': 'R.southwardae',
 'V_diagona': 'V.diagonalis',
 'V_extenta': 'V.extenta',
 'V_gigas1_': 'V.gigas1',
 'V_gigas2_': 'V.gigas2',
 'V_marissi': 'V.marissinica',
 'V_okutani': 'V.okutanii',
 'V_soyoae1': 'V.soyoae1',
 'V_soyoae2': 'V.soyoae2'}

fl=['Bathy_MS2', 'SUP05_SP6']
ruthia=['R_fausta_', 'R_magnifi', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa']
gigas=['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_', 'V_marissi', 'V_okutani', 'V_soyoae1', 'V_soyoae2']
partitions={'a':[set(fl),set(branches_dic.keys())-set(fl)],
    'b':[set(ruthia),set(branches_dic.keys())-set(ruthia)],
    'c':[set(gigas),set(branches_dic.keys())-set(gigas)],
    'd':[set(['V_okutani', 'V_soyoae1', 'V_soyoae2']),set(branches_dic.keys())-set(['V_okutani', 'V_soyoae1', 'V_soyoae2'])],
    'e':[set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_'])],
    'f':[set(['V_diagona', 'V_extenta']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta'])],
    'g':[set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa'])],
    'h':[set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_rectima'])],
    'i':[set(['R_fausta_', 'R_pacific', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_rectima'])],
    'j':[set(['R_pliocar', 'R_southwa']),set(branches_dic.keys())-set(['R_pliocar', 'R_southwa'])],
    'k':[set(['V_soyoae1', 'V_soyoae2']),set(branches_dic.keys())-set(['V_soyoae1', 'V_soyoae2'])],
    'l':[set(['V_gigas1_', 'V_gigas2_']),set(branches_dic.keys())-set(['V_gigas1_', 'V_gigas2_'])],
    'm':[set(['V_diagona', 'V_extenta', 'V_marissi']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta', 'V_marissi'])],
    'n':[set(['R_fausta_', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_rectima'])]}

def find_node_partition(br,tree_str):
    t = Tree(tree_str+';', format=1) # load ete3 trees from tree string
#     sup05=[l for l in t.get_leaf_names() if 'SUP05' in l][0]
#     rmag=[l for l in t.get_leaf_names() if 'R_mag' in l][0]

#     bathy=[l for l in t.get_leaf_names() if 'MS2017' in l][0]
#     ancestor = t.get_common_ancestor(sup05,bathy)
#     t.set_outgroup(ancestor) # set ancestor of Sup05 & Bathy as root
#     t.set_outgroup(sup05) # set ancestor of Sup05
#     t.set_outgroup(rmag) # set ancestor of Rmag
    t.unroot()

    node=t&br # go to node in interest
    leaves=set([l[:9] for l in node.get_leaf_names()]) # get all leaves for that node    
    category='NA'
    for k,v in partitions.items(): # find category associated to that leaves set
        if leaves in v:
            category=k
            break
    if category=='NA': # in tree does not follow main, topology, try to root the tree by ancestor of free-living
        sup05=[l for l in t.get_leaf_names() if 'SUP05' in l][0]
        bathy=[l for l in t.get_leaf_names() if 'MS2017' in l][0]
        ancestor = t.get_common_ancestor(sup05,bathy)
        t.set_outgroup(ancestor) # set ancestor of Sup05 & Bathy as root
        node=t&br # go to node in interest
        leaves=set([l[:9] for l in node.get_leaf_names()]) # get all leaves for that node    
        for k,v in partitions.items(): # find category associated to that leaves set
            if leaves in v:
                category=k
                break
    if category=='NA': # if category still unresolved: try to assess broader category
        upleaves=sorted(list(set([l[:1] for l in node.get_ancestors()[0].get_leaf_names()])))
        if len(upleaves)==1:
            category='NA_'+str(upleaves[0])
#             print(category)

        elif upleaves==['R','V']:
            category='NA_S'
#             print(category)
        else: # in category still unresolved: inspect
            category='NA'
            print(br, leaves, node.get_ancestors()[0].name)
            print(t.get_ascii(show_internal=True))

    return(category)

df=pd.DataFrame([],columns=['gene_id']+list(partitions.keys())+list(branches_dic.values())+['NA_R','NA_V','NA_S','NA'])

#####################

branches=[]
for gene in genes:
#     print(gene)
    remote_file = open(path+'/'+gene+'.absrel.json')
    absrel_json = json.load(remote_file)
    if pval_threshold==0.05:
        if absrel_json['test results']["positive test results"]==0:
            continue
    new_row={'gene_id':gene}
    for branch in absrel_json['branch attributes']['0'].keys():
        branches+=[branch]
        branch_pval=absrel_json['branch attributes']['0'][branch]['Corrected P-value']
        if branch_pval<pval_threshold:
            br=branch[:9]
#             print(br)
            if br[:4]=='Node':
                tree_str=absrel_json['input']['trees']['0']
                cat=find_node_partition(br,tree_str)
                if cat=='NA':print(gene)
                new_row[cat]=1
            else:
                cat=branches_dic[br]
                new_row[cat]=1
    if len(new_row.keys())>1:
        df=df.append(new_row,ignore_index=True)

####### Edit table to add different combinations of categories
df=df.fillna(0)
df['All']=df.sum(axis=1)
df['Free living']=df[['Bathy','SUP05']].sum(axis=1)
df['Gigas']=df['NA_V']+df[['d', 'e', 'f', 'k', 'l','m','V.diagonalis', 'V.extenta',
               'V.gigas1', 'V.gigas2', 'V.marissinica','V.okutanii', 'V.soyoae1',
               'V.soyoae2']].sum(axis=1)
df['Ruthia']=df['NA_R']+df[['g', 'h', 'i', 'j', 'n', 'R.fausta', 'R.magnifica', 'R.pacifica',
       'R.phaseoliformis', 'R.pliocardia', 'R.rectimargo', 'R.southwardae']].sum(axis=1)
df['Symbionts']=df['NA_S']+df[['Gigas','Ruthia','b','c']].sum(axis=1)
df['All non terminal branches']=df[['NA_V','NA_R','NA_S','NA','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                                   'k', 'l','m', 'n']].sum(axis=1) # selection on branches including these that are not part of main tree
df['All groups']=df[['Gigas','Ruthia','Free living']].sum(axis=1)
df=df[['gene_id','All','All groups','Symbionts','Gigas','Ruthia','Free living','All non terminal branches',
      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 
      'Bathy', 'SUP05', 'R.fausta', 'R.magnifica', 'R.pacifica',
       'R.phaseoliformis', 'R.pliocardia', 'R.rectimargo', 'R.southwardae',
       'V.diagonalis', 'V.extenta', 'V.gigas1', 'V.gigas2', 'V.marissinica',
       'V.okutanii', 'V.soyoae1', 'V.soyoae2','NA_V','NA_R','NA_S','NA']]



Node5 {'Bathy_MS2', 'R_pacific', 'R_rectima', 'R_pliocar', 'R_fausta_'} Node4

             /-Bathy_MS2017_2063_unknown_description_
            |
            |               /-R_fausta_HUE58_RS06145_unknown_description_
        /Node5         /Node9
       |    |     /Node8    \-R_rectimargo_Rrec_peg_807_unknown_description_
       |    |    |    |
   /Node4    \Node7    \-R_pacifica_Rpac_peg_116_unknown_description_
  |    |         |
  |    |          \-R_pliocardia_Rpli_peg_1468_unknown_description_
  |    |
  |     \-SUP05_SP60_04550_unknown_description_
--|
  |           /-R_magnifica_Rmag_0079_unknown_description_
  |     /Node15
  |    |     |      /-R_phaseoliformis_Rpha_peg_2125_unknown_description_
  |    |      \Node17
  |    |            \-R_southwardae_Rsou_peg_1659_unknown_description_
   \Node3
       |           /-V_gigas1_Vgig1_peg_92_unknown_description_
       |     /Node20
       |    |      \-V_gigas2_Vgig2_peg_878_unknown_description_
       |    |
        \Node2

In [9]:
df.sum() # raw

gene_id                      Rmag_0002Rmag_0006Rmag_0010Rmag_0015Rmag_0018R...
All                                                                        155
All groups                                                                 107
Symbionts                                                                  101
Gigas                                                                       14
Ruthia                                                                      67
Free living                                                                 26
All non terminal branches                                                   87
a                                                                           26
b                                                                           13
c                                                                            6
d                                                                            0
e                                                   

In [10]:
df.to_csv(path_out+'absrel_results_pval_'+str(pval_threshold)+'.txt',header=True,index=False,sep='\t')